In [1]:
import gradio as gr
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt


In [2]:

# Load your dataset into a DataFrame (replace 'your_dataset.csv' with your file path)
df_cnn = pd.read_csv('dataset_cnn.csv')
df_svm = pd.read_csv('dataset_svm.csv')
df_rf = pd.read_csv('dataset_rf.csv')
# Function to filter data, generate confusion matrix, and calculate accuracy


In [3]:
df_cnn

,index,Modulation Type,SNR,data,labels,prediction
0,1,BPSK,6,"[[-1.9263707406534007, 0.9448954682334088, 0.1...",0,0
1,3,PAM4,-4,"[[-0.4770900364771389, -2.492696312787814, -4....",1,1
2,131079,QAM64,12,"[[-9.968040818769326, -3.5062374914649093, 6.8...",1,1
3,131080,BPSK,16,"[[1.2044663480237794, 0.7242794811175602, 0.54...",0,0
4,10,GFSK,-14,"[[-2.3472207328072563, 8.12814152549767, 25.91...",0,1
...,...,...,...,...,...,...
48535,131055,GFSK,-4,"[[1.2417729567105766, 0.08593597030070695, -2....",0,0
48536,131057,GFSK,14,"[[1.468274700690957, 1.7381923305166898, 1.912...",1,1
48537,131061,BPSK,16,"[[0.33757994899447413, 0.010285189837328167, -...",0,0
48538,131064,GFSK,6,"[[-0.0469691179743148, 0.36828529449728953, 0....",1,1


In [4]:
df_svm

,Modulation Type,SNR,data,labels,prediction
0,8PSK,-14,"[[6.132419790780786, -15.053748996403085, 17.8...",1,1
1,BPSK,-18,"[[19.11433257202686, 33.96462572651337, 61.550...",0,1
2,GFSK,-14,"[[-1.3499705263522184, 3.904487412455078, 41.9...",0,0
3,8PSK,-12,"[[-19.61448871120961, 37.2608893508103, -27.72...",1,1
4,PAM4,0,"[[-0.010739661105336529, -4.7096615735552305, ...",1,1
...,...,...,...,...,...
48535,8PSK,-20,"[[13.23203283759484, 53.00921090494359, -40.50...",1,1
48536,BPSK,8,"[[-1.7927829016068015, -1.5710195447786792, -0...",0,1
48537,BPSK,-16,"[[11.843756501383302, -31.388275019801068, 11....",0,0
48538,BPSK,-12,"[[-1.5697971086408224, 17.74682350326093, 4.29...",0,1


In [5]:
df_rf

,Modulation Type,SNR,data,labels,prediction
0,8PSK,-14,"[[6.132419790780786, -15.053748996403085, 17.8...",1,1
1,BPSK,-18,"[[19.11433257202686, 33.96462572651337, 61.550...",0,1
2,GFSK,-14,"[[-1.3499705263522184, 3.904487412455078, 41.9...",0,1
3,8PSK,-12,"[[-19.61448871120961, 37.2608893508103, -27.72...",1,1
4,PAM4,0,"[[-0.010739661105336529, -4.7096615735552305, ...",1,1
...,...,...,...,...,...
48535,8PSK,-20,"[[13.23203283759484, 53.00921090494359, -40.50...",1,1
48536,BPSK,8,"[[-1.7927829016068015, -1.5710195447786792, -0...",0,0
48537,BPSK,-16,"[[11.843756501383302, -31.388275019801068, 11....",0,1
48538,BPSK,-12,"[[-1.5697971086408224, 17.74682350326093, 4.29...",0,1


In [3]:
def predict_from_iq(model_type,modulation, snr):
    # Load the dataset based on the selected model type
    if model_type == "CNN":
        filtered_data = df_cnn[(df_cnn['Modulation Type'] == modulation) & (df_cnn['SNR'] == snr)]
    elif model_type == "SVM":
        filtered_data = df_svm[(df_svm['Modulation Type'] == modulation) & (df_svm['SNR'] == snr)]
    elif model_type == "RF":
        filtered_data = df_rf[(df_rf['Modulation Type'] == modulation) & (df_rf['SNR'] == snr)]
    else:
        return "Invalid model type selected.", None

    # Check if there is data available for the selected modulation and SNR
    if filtered_data.empty:
        return "No data available for the selected Modulation and SNR.", None

    # Get the true labels and predictions
    true_labels = filtered_data['labels']
    predictions = filtered_data['prediction']
  
    # Generate the confusion matrix
    conf_matrix = confusion_matrix(true_labels, predictions)
    
    # Generate the classification report
    report = classification_report(true_labels, predictions, target_names=['Noise', 'Signal'], output_dict=True)
    accuracy = report['accuracy']
    
    # Plot the confusion matrix
    fig, ax = plt.subplots()
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', cbar=False, ax=ax, linewidths=0.5, linecolor='black',
                annot_kws={"size": 14})  # Increase annotation size

    # Adjust text color based on heatmap intensity
    for t in ax.texts:
        val = int(t.get_text())
        t.set_color('white' if val > conf_matrix.max() / 2 else 'black')
    
    plt.xlabel('Predicted Label', fontsize=14)
    plt.ylabel('True Label', fontsize=14)
    plt.title(f"Confusion Matrix for {modulation} at SNR {snr} dB ({model_type})", fontsize=16)
    
    plt.tight_layout()
    fig.savefig("conf_matrix.png")
  
    return f"Accuracy: {accuracy * 100:.2f}%", "conf_matrix.png"


In [4]:
interface = gr.Interface(
    fn=predict_from_iq,
    inputs=[
        gr.Dropdown(["CNN", "SVM", "RF"], label="Select Model"),
        gr.Dropdown(label="Select Modulation Type", choices=['BPSK', 'GFSK', '8PSK', 'PAM4', 'QAM16', 'QAM64']),
        gr.Dropdown(label="Select SNR Value", choices=[-20, -16, -12, -10, -8, -6, -4, -2, 0, 2, 4, 6, 8, 10, 12, 14, 16])
    ],
    outputs=["text", "image"],
    title="Signal Prediction Accuracy and Confusion Matrix"
)

# Launch the Gradio interface
interface.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


IMPORTANT: You are using gradio version 3.50.2, however version 4.44.1 is available, please upgrade.
--------
